<h1>1장 대규모 언어 모델 소개</h1>
<i>흥미진진한 언어 AI 분야를 탐험합니다.</i>


<a href="https://github.com/rickiepark/handson-llm"><img src="https://img.shields.io/badge/GitHub%20Repository-black?logo=github"></a>
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rickiepark/handson-llm/blob/main/chapter01.ipynb)

---

이 노트북은 <[핸즈온 LLM](https://tensorflow.blog/handson-llm/)> 책 1장의 코드를 담고 있습니다.

---

<a href="https://tensorflow.blog/handson-llm/">
<img src="https://tensorflow.blog/wp-content/uploads/2025/05/ed95b8eca688ec98a8_llm.jpg" width="350"/></a>


---

💡 **NOTE**: 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.

---

## Hugging Face Transformers로 사전학습 언어 모델(Phi-3)을 불러와 텍스트를 생성하는 가장 기본적인 LLM 활용 실습

In [ ]:
# -----------------------------------------------------------
# 1. Hugging Face Transformers의 로깅(logging) 기능 불러오기
#    - 모델을 다운로드하거나 텍스트를 생성할 때
#      화면에 '진행 표시줄(progress bar)'이 나타납니다.
#    - 하지만 GitHub, VS Code, 또는 Jupyter 환경에서는
#      이 진행 표시줄이 깨지거나 보이지 않아 에러처럼 보일 수 있습니다.
# -----------------------------------------------------------
from transformers.utils import logging

# -----------------------------------------------------------
# 2. 진행 표시줄 끄기
#    - logging.disable_progress_bar()를 사용하면
#      모델 실행 중에 나타나는 프로그레스 바를 숨길 수 있습니다.
#    - 화면이 깔끔해지고 출력 오류가 줄어듭니다.
# -----------------------------------------------------------
logging.disable_progress_bar()

# Phi-3

첫 번째 단계로 빠른 추론을 위해 모델을 GPU에 로드합니다. 모델과 토크나이저를 각각 로드합니다(항상 이렇게 할 필요는 없습니다).

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# ---------------------------------------------------------
# 1. 사용할 모델과 토크나이저 클래스를 불러옵니다.
#    - AutoModelForCausalLM : "문장을 이어서 생성"하는 모델을 로드할 때 사용
#    - AutoTokenizer        : 문장을 숫자로 바꾸고(토큰화), 다시 문장으로 바꾸는 역할
# ---------------------------------------------------------


# ---------------------------------------------------------
# 2. 사전학습된 언어 모델을 불러옵니다.
#    - "microsoft/Phi-3-mini-4k-instruct" 라는 이름의 모델을 다운로드하고 메모리에 올립니다.
#    - device_map="cuda" : 그래픽카드(GPU)를 사용해 모델을 실행하겠다는 뜻
#    - dtype="auto"      : 모델이 알아서 적절한 데이터 타입(정밀도)을 선택하게 함
# ---------------------------------------------------------
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",  # GPU 사용
    dtype="auto",       # 모델이 자동으로 최적 데이터 타입 선택
)


# ---------------------------------------------------------
# 3. 토크나이저를 불러옵니다.
#    - 같은 이름의 모델용 토크나이저를 가져옵니다.
#    - 토크나이저는 사람이 읽는 문장을 모델이 이해할 수 있는 '숫자(토큰)'로 바꿔줍니다.
#    - 모델의 출력(숫자)을 다시 자연어 문장으로 되돌릴 때도 사용됩니다.
# ---------------------------------------------------------
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


모델과 토크나이저를 직접 사용할 수 있지만 `pipeline` 함수를 사용하는 것이 훨씬 쉽습니다.

- pipeline() → 모델 사용을 쉽게 만들어주는 자동 연결 장치

- "text-generation" → “문장을 만들어 주세요” 작업을 선택하는 것

- generator("문장") → 이어지는 텍스트를 만들어줌

- do_sample=False → 랜덤 없이 안정적·일관된 답변

- max_new_tokens → 생성되는 문장의 최대 길이 설정

In [ ]:
from transformers import pipeline

# -----------------------------------------------------------
# 1. 텍스트 생성 파이프라인 만들기
#    pipeline() 함수는 모델과 토크나이저를 연결해서
#    쉽게 사용할 수 있는 “텍스트 생성기”를 만들어 줍니다.
#
#    예를 들어,
#    generator("오늘 날씨는")  → 이어지는 문장을 자동으로 생성해 줍니다.
# -----------------------------------------------------------
generator = pipeline(
    "text-generation",  # 어떤 작업을 할지 선택 → 여기서는 '텍스트 생성'
    model=model,        # 앞에서 불러온 언어 모델 사용
    tokenizer=tokenizer,# 문장 ↔ 숫자(토큰) 변환을 담당하는 토크나이저
    return_full_text=False, # 입력 문장을 결과에 다시 포함하지 않도록 설정
    max_new_tokens=500, # 모델이 새로 만들어낼 최대 글자 수(토큰 수)
    do_sample=False     # 랜덤이 아닌 '가장 가능성이 높은 결과'만 선택
)

Device set to use cuda
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


마지막으로 프롬프트를 작성하고 모델에 주입합니다.

- messages = 모델에게 줄 질문
- generator(messages) = 모델이 답변 생성
- output[0]["generated_text"] = 만들어진 최종 문장

In [ ]:
# -----------------------------------------------------------
# 1. 프롬프트(사용자 요청) 만들기
#    - 모델에게 "어떤 문장을 만들어 달라"라고 요청하는 부분입니다.
#    - messages 리스트 안에 role과 content를 넣는 구조는
#      '대화형 모델(ChatGPT 스타일)'과 비슷한 형식입니다.
# -----------------------------------------------------------
messages = [
    {"role": "user", "content": "Create a funny joke about chickens."}
    # role: "user" → 사람이 한 말
    # content: 실제 요청 내용 (치킨에 대한 농담 만들어줘)
]

# -----------------------------------------------------------
# 2. 생성기(generator)를 이용해서 텍스트를 만들어내기
#    - generator(messages)를 호출하면
#      모델이 messages 내용을 바탕으로 이어지는 문장을 만듭니다.
# -----------------------------------------------------------
output = generator(messages)

# -----------------------------------------------------------
# 3. 생성된 문장 출력하기
#    - output은 리스트 형태로 나오며,
#      그 중 첫 번째 결과의 "generated_text"에 생성된 문장이 들어 있습니다.
# -----------------------------------------------------------
print(output[0]["generated_text"])

 Why did the chicken join the band? Because it had the drumsticks!


In [6]:
# -----------------------------------------------------------
# 1. 프롬프트(사용자 요청) 만들기
#    - 모델에게 "어떤 문장을 만들어 달라"라고 요청하는 부분입니다.
#    - messages 리스트 안에 role과 content를 넣는 구조는
#      '대화형 모델(ChatGPT 스타일)'과 비슷한 형식입니다.
# -----------------------------------------------------------
messages = [
    {"role": "user", "content": "치킨에 대한 농담 만들어줘"}
    # role: "user" → 사람이 한 말
    # content: 실제 요청 내용 (치킨에 대한 농담 만들어줘)
]

# -----------------------------------------------------------
# 2. 생성기(generator)를 이용해서 텍스트를 만들어내기
#    - generator(messages)를 호출하면
#      모델이 messages 내용을 바탕으로 이어지는 문장을 만듭니다.
# -----------------------------------------------------------
output = generator(messages)

# -----------------------------------------------------------
# 3. 생성된 문장 출력하기
#    - output은 리스트 형태로 나오며,
#      그 중 첫 번째 결과의 "generated_text"에 생성된 문장이 들어 있습니다.
# -----------------------------------------------------------
print(output[0]["generated_text"])

 치킨 농담:

"치킨 먹기 전에 먹을 때 치킨 배추를 먹어야 치킨이 더 맛있어요. 그럼 먹을 때 배추를 먹으면 치킨이 더 맛있어요."


In [7]:
# -----------------------------------------------------------
# 1. 프롬프트(사용자 요청) 만들기
#    - 모델에게 "어떤 문장을 만들어 달라"라고 요청하는 부분입니다.
#    - messages 리스트 안에 role과 content를 넣는 구조는
#      '대화형 모델(ChatGPT 스타일)'과 비슷한 형식입니다.
# -----------------------------------------------------------
messages = [
    {"role": "user", "content": "오늘 날씨가 어때?"}
    # role: "user" → 사람이 한 말
    # content: 실제 요청 내용
]

# -----------------------------------------------------------
# 2. 생성기(generator)를 이용해서 텍스트를 만들어내기
#    - generator(messages)를 호출하면
#      모델이 messages 내용을 바탕으로 이어지는 문장을 만듭니다.
# -----------------------------------------------------------
output = generator(messages)

# -----------------------------------------------------------
# 3. 생성된 문장 출력하기
#    - output은 리스트 형태로 나오며,
#      그 중 첫 번째 결과의 "generated_text"에 생성된 문장이 들어 있습니다.
# -----------------------------------------------------------
print(output[0]["generated_text"])

 저는 2023년 8월 10일 오후 3시에 답변하겠습니다. 현재 서울 지역의 날씨는 평상으로 약 25℃의 기온이 있고, 풍속은 10m/s로 있습니다. 하늘은 푸른색이고 비가 없습니다. 날씨 정보는 당신의 지역에 따라 조금씩 달라질 수 있으므로, 최신 지역 별 날씨 정보를 알려주세요.
